In [40]:

# Import All Necessary Modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3


# Load the Referenced DataFrames from CSV Files
# Replace the file paths with your actual paths
df_bikes = pd.read_csv('df_bikes.csv')
df_yelp_unique = pd.read_csv('df_yelp_unique.csv')
df_fs = pd.read_csv('df_foursquare.csv')

In [43]:
df_fs

,Unnamed: 0,name,category,city,region,country,latitude,longitude,distance,search_latitude,search_longitude
0,0,Leslieville Farmers Market,Farmers Market,Toronto,ON,CA,43.664679,-79.319687,40,43.665269,-79.319796
1,1,McDonald's,Burger Joint,Toronto,ON,CA,43.665065,-79.319000,62,43.665269,-79.319796
2,2,T.O. Ceviche Bar,Peruvian Restaurant,Toronto,ON,CA,43.665973,-79.318863,108,43.665269,-79.319796
3,3,Chalet Beauty Bar,Hair Salon,Toronto,ON,CA,43.665973,-79.318863,108,43.665269,-79.319796
4,4,Chick-N-Joy,Diner,Toronto,ON,CA,43.665121,-79.321347,129,43.665269,-79.319796
...,...,...,...,...,...,...,...,...,...,...,...
32155,32155,Tim Choi Chinese Food,Chinese Restaurant,Scarborough,ON,CA,43.745400,-79.220350,849,43.745311,-79.209707
32156,32156,Tropics Restaurant and Bar,Restaurant,Scarborough,ON,CA,43.745400,-79.220350,854,43.745311,-79.209707
32157,32157,Join Sushi,Sushi Restaurant,Toronto,ON,CA,43.738910,-79.217607,951,43.745311,-79.209707
32158,32158,Domino's Pizza,Pizzeria,Toronto,ON,CA,43.738878,-79.218070,967,43.745311,-79.209707


In [31]:
ls

Untitled.ipynb                         joining_data.ipynb
city_bikes.ipynb                       joining_data_final_spatial_join.ipynb
df_bikes.csv                           joining_data_updated.ipynb
df_foursquare.csv                      model_building.ipynb
df_yelp_duplicates.csv                 networks.csv
df_yelp_unique.csv                     yelp_foursquare_EDA.ipynb


Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [34]:
# Load the Referenced DataFrames from CSV Files
# Replace the file paths with your actual paths
df_bikes = pd.read_csv('df_bikes.csv')
df_yelp_unique = pd.read_csv('df_yelp_unique.csv')
df_fs = pd.read_csv('df_foursquare.csv')

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [44]:
# Correct column names for latitude and longitude in each DataFrame
bikes_lat, bikes_long = 'latitude', 'longitude'
yelp_lat, yelp_long = 'coordinates.latitude', 'coordinates.longitude'
fs_lat, fs_long = 'latitude', 'longitude'

# Function to find the nearest POI for each bike station
def find_nearest_poi(df1, df2, prefix, lat1, long1, lat2, long2):
    closest_pois = []
    for i, row1 in df1.iterrows():
        min_distance = float('inf')
        closest_poi = None
        for j, row2 in df2.iterrows():
            distance = ((row1[lat1] - row2[lat2]) ** 2 + (row1[long1] - row2[long2]) ** 2) ** 0.5
            if distance < min_distance:
                min_distance = distance
                closest_poi = row2
        closest_pois.append(closest_poi.add_prefix(prefix + '_'))
    return pd.DataFrame(closest_pois)

# Find the nearest Yelp and Foursquare POIs for each bike station
nearest_yelp = find_nearest_poi(df_bikes, df_yelp_unique, 'yelp', bikes_lat, bikes_long, yelp_lat, yelp_long)
nearest_fs = find_nearest_poi(df_bikes, df_fs, 'fs', bikes_lat, bikes_long, fs_lat, fs_long)

# Join the nearest Yelp and Foursquare data with the bike stations data
joined_data = pd.concat([df_bikes, nearest_yelp, nearest_fs], axis=1)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.

In [39]:

# Perform Spatial Join Based on Latitude and Longitude

# Function to find the nearest POI for each bike station
def find_nearest_poi(df1, df2, prefix):
    closest_pois = []
    for i, row1 in df1.iterrows():
        min_distance = float('inf')
        closest_poi = None
        for j, row2 in df2.iterrows():
            distance = ((row1['latitude'] - row2['latitude']) ** 2 + (row1['longitude'] - row2['longitude']) ** 2) ** 0.5
            if distance < min_distance:
                min_distance = distance
                closest_poi = row2
        closest_pois.append(closest_poi.add_prefix(prefix + '_'))
    return pd.DataFrame(closest_pois)

# Find the nearest Yelp and Foursquare POIs for each bike station
nearest_yelp = find_nearest_poi(df_bikes, df_yelp_unique, 'yelp')
nearest_fs = find_nearest_poi(df_bikes, df_fs, 'fs')

# Join the nearest Yelp and Foursquare data with the bike stations data
joined_data = pd.concat([df_bikes, nearest_yelp, nearest_fs], axis=1)

# The resulting dataframe 'joined_data' contains the bike stations data enhanced with the nearest Yelp and Foursquare POIs


KeyError: 'latitude'

In [ ]:

# Import necessary libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Visualization for EDA
# For demonstration, let's visualize the distribution of Yelp ratings around bike stations

# Ensure that the 'rating' column does not have NaN values
joined_data_yelp = joined_data_yelp.dropna(subset=['rating'])

# Create the plot
plt.figure(figsize=(10, 6))
sns.boxplot(x='station_id', y='rating', data=joined_data_yelp)
plt.xticks(rotation=90)
plt.title('Distribution of Yelp Ratings Around Bike Stations')
plt.xlabel('Station ID')
plt.ylabel('Yelp Rating')
plt.show()


In [ ]:

# Import SQLite3 library
import sqlite3

# Create or connect to a database
conn = sqlite3.connect('joined_data.db')

# Store the joined data in the SQLite3 database
joined_data_yelp.to_sql('joined_data_yelp', conn, if_exists='replace', index=False)
joined_data_fs.to_sql('joined_data_fs', conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

# The data is now stored in 'joined_data.db' with tables 'joined_data_yelp' and 'joined_data_fs'


In [ ]:

# Data Validation Before and After Join

# Validate the data before the join
print("Data Validation Before Join")
print("Number of rows in df_bikes:", len(df_bikes))
print("Number of rows in df_yelp_unique:", len(df_yelp_unique))
print("Number of rows in df_fs:", len(df_fs))

# Validate the data after the join
print("\nData Validation After Join")
print("Number of rows in joined_data_yelp:", len(joined_data_yelp))
print("Number of rows in joined_data_fs:", len(joined_data_fs))

# Additional checks can include verifying unique identifiers, checking for missing values, etc.
